In [1]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('LeetCode').getOrCreate()

24/12/25 20:27:51 WARN Utils: Your hostname, madiv resolves to a loopback address: 127.0.1.1; using 192.168.1.11 instead (on interface wlo1)
24/12/25 20:27:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/25 20:27:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark_Sales = spark.read.option('header', True).option('delimiter', ',').csv('Sales.csv')
spark_Product = spark.read.option('header', True).option('delimiter', ',').csv('Product.csv')

pandas_Sales = pd.read_csv('Sales.csv')
pandas_Product = pd.read_csv('Product.csv')

spark_Sales.show()
spark_Product.show()

print(pandas_Sales.head())
print(pandas_Product.head())

+-------+----------+----+--------+-----+
|sale_id|product_id|year|quantity|price|
+-------+----------+----+--------+-----+
|      1|       100|2008|      10| 5000|
|      2|       100|2009|      12| 5000|
|      7|       200|2011|      15| 9000|
+-------+----------+----+--------+-----+

+----------+------------+
|product_id|product_name|
+----------+------------+
|       100|       Nokia|
|       200|       Apple|
|       300|     Samsung|
+----------+------------+

   sale_id  product_id  year  quantity  price
0        1         100  2008        10   5000
1        2         100  2009        12   5000
2        7         200  2011        15   9000
   product_id product_name
0         100        Nokia
1         200        Apple
2         300      Samsung


### SQL

In [3]:
def sales_analysis(sales: pyspark.sql.dataframe.DataFrame, 
                   product: pyspark.sql.dataframe.DataFrame) -> pyspark.sql.dataframe.DataFrame:
    sales.createOrReplaceTempView('Sales')
    product.createOrReplaceTempView('Product')
    
    sqlQuery = \
    '''
    SELECT product_name, year, price FROM Product
    INNER JOIN Sales ON
    Product.product_id = Sales.product_id
    '''
    
    output = spark.sql(sqlQuery = sqlQuery)
    return output

output = sales_analysis(spark_Sales, spark_Product)
output.show()

+------------+----+-----+
|product_name|year|price|
+------------+----+-----+
|       Nokia|2008| 5000|
|       Nokia|2009| 5000|
|       Apple|2011| 9000|
+------------+----+-----+



### PySpark

In [4]:
def sales_analysis(sales: pyspark.sql.dataframe.DataFrame, 
                   product: pyspark.sql.dataframe.DataFrame) -> pyspark.sql.dataframe.DataFrame:
    output = sales.join(product, on = 'product_id', how = 'inner')\
                    .select(['product_name', 'year', 'price'])
    return output

output = sales_analysis(spark_Sales, spark_Product)
output.show()

+------------+----+-----+
|product_name|year|price|
+------------+----+-----+
|       Nokia|2008| 5000|
|       Nokia|2009| 5000|
|       Apple|2011| 9000|
+------------+----+-----+



### Pandas

In [5]:
def sales_analysis(sales: pd.DataFrame, product: pd.DataFrame) -> pd.DataFrame:
    output = sales.merge(product, on = 'product_id', how = 'inner')
    output = output[['product_name', 'year', 'price']]
    return output

output = sales_analysis(pandas_Sales, pandas_Product)
output.head()

,product_name,year,price
0,Nokia,2008,5000
1,Nokia,2009,5000
2,Apple,2011,9000
